In [1]:
# app/src/api/main.py
from fastapi import FastAPI, HTTPException
from app.src.features.preprocessing import Preprocessing
from app.src.utils.utils import predicted_values
from app.src.utils.schemas import ItemsFullSchema
from app.src.config import MODEL_PATH, MODEL_NAME
import pandas as pd
import os
from joblib import load

app = FastAPI(title="API for predicted model for New or Used items", version="1.0")

# Cargar modelo
model_loaded = os.path.join(MODEL_PATH, MODEL_NAME)
try:
    model = load(model_loaded)
except Exception as e:
    raise RuntimeError(f"Error cargando el modelo: {e}")

preprocessor = Preprocessing()

In [ ]:
# {
#   "inputs": [
#     {
#       "seller_address": {
#         "country": {"name": "Argentina", "id": "AR"},
#         "state": {"name": "Capital Federal", "id": "AR-C"},
#         "city": {"name": "San Cristóbal", "id": "TUxBQlNBTjkwNTZa"}
#       },
#       "warranty": null,
#       "condition": "new",
#       "base_price": 80.0,
#       "price": 80.0,
#       "shipping": {
#         "local_pick_up": true,
#         "free_shipping": false,
#         "mode": "not_specified",
#         "tags": []
#       },
#       "non_mercado_pago_payment_methods": [
#         {"description": "Transferencia bancaria", "id": "MLATB", "type": "G"},
#         {"description": "Acordar con el comprador", "id": "MLAWC", "type": "G"},
#         {"description": "Efectivo", "id": "MLAMO", "type": "G"}
#       ],
#       "seller_id": 74952096,
#       "listing_type_id": "bronze",
#       "available_quantity": 1,
#       "sold_quantity": 0.0,
#       "start_time": 1441485773000,
#       "stop_time": 1446669773000,
#       "accepts_mercadopago": true,
#       "currency_id": "ARS",
#       "tags": ["dragged_bids_and_visits"],
#       "automatic_relist": false,
#       "category_id": "MLA126406",
#       "parent_item_id": "MLA568261029",
#       "status": "active",
#       "video_id": null,
#       "original_price": null,
#       "buying_mode": "buy_it_now",
#       "initial_quantity": 1
#     }
#   ]
# }

sample_input = {
    "inputs": [
        {
            "seller_address": {
                "country": {"name": "Argentina", "id": "AR"},
                "state": {"name": "Capital Federal", "id": "AR-C"},
                "city": {"name": "San Cristóbal", "id": "TUxBQlNBTjkwNTZa"}
            },
            "warranty": None,
            "base_price": 80.0,
            "price": 80.0,
            "shipping": {
                "local_pick_up": True,
                "free_shipping": False,
                "mode": "not_specified",
                "tags": []
            },
            "non_mercado_pago_payment_methods": [
                {"description": "Transferencia bancaria", "id": "MLATB", "type": "G"},
                {"description": "Acordar con el comprador", "id": "MLAWC", "type": "G"},
                {"description": "Efectivo", "id": "MLAMO", "type": "G"}
            ],
            "seller_id": 74952096,
            "listing_type_id": "bronze",
            "available_quantity": 1,
            "sold_quantity": 0.0,
            "start_time": 1441485773000,
            "stop_time": 1446669773000,
            "accepts_mercadopago": True,
            "currency_id": "ARS",
            "tags": ["dragged_bids_and_visits"],
            "automatic_relist": False,
            "category_id": "MLA126406",
            "parent_item_id": "MLA568261029",
            "status": "active",
            "video_id": None,
            "original_price": None,
            "buying_mode": "buy_it_now",
            "initial_quantity": 1
        }
    ]
}

In [5]:
df = pd.DataFrame(sample_input["inputs"])
df

,seller_address,warranty,base_price,price,shipping,non_mercado_pago_payment_methods,seller_id,listing_type_id,available_quantity,sold_quantity,...,currency_id,tags,automatic_relist,category_id,parent_item_id,status,video_id,original_price,buying_mode,initial_quantity
0,"{'country': {'name': 'Argentina', 'id': 'AR'},...",None,80.0,80.0,"{'local_pick_up': True, 'free_shipping': False...","[{'description': 'Transferencia bancaria', 'id...",74952096,bronze,1,0.0,...,ARS,[dragged_bids_and_visits],False,MLA126406,MLA568261029,active,None,None,buy_it_now,1


In [6]:
preprocessor = Preprocessing()
processed = preprocessor.transform(df)
processed = processed.drop(columns=['condition'])
processed

,initial_quantity,listing_type_id,seller_city,price,available_quantity,week_day,sold_quantity,seller_state,garantia_aplica,shipping_mode,month_start,month_stop,tarjeta_de_credito,dragged_bids_and_visits,transferencia_bancaria,shipping_local_pick_up,efectivo,automatic_relist,days_active,acordar_con_el_comprador
0,1,0,0,80.0,1,0,NaN,0,0,0,0,0,False,1,True,True,True,False,0,True


In [7]:
# Cargar modelo
model_loaded = os.path.join(MODEL_PATH, MODEL_NAME)
try:
    model = load(model_loaded)
except Exception as e:
    raise RuntimeError(f"Error cargando el modelo: {e}")

In [8]:
processed.columns

Index(['initial_quantity', 'listing_type_id', 'seller_city', 'price',
       'available_quantity', 'week_day', 'sold_quantity', 'seller_state',
       'garantia_aplica', 'shipping_mode', 'month_start', 'month_stop',
       'tarjeta_de_credito', 'dragged_bids_and_visits',
       'transferencia_bancaria', 'shipping_local_pick_up', 'efectivo',
       'automatic_relist', 'days_active', 'acordar_con_el_comprador'],
      dtype='object')

In [9]:
preds = model.predict(processed)

In [10]:
predicted_values(preds)

'New'